In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import keras 
%matplotlib inline

In [2]:
from keras.optimizers import Adam 
from keras import Sequential,Model

#A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor

#The core idea of Sequential API is simply arranging the Keras layers in a sequential order and so, it is called Sequential API.
from keras.layers import Dense,BatchNormalization,LeakyReLU,Input,Reshape,Dropout,Flatten,Activation

# Dense Layer is fully connected layer ,i.e hidden layer 
# BatchNormalization is a technique to normalize the output from previous layer 
# LeakyRel is a activation function 
# Reshape , it reshapes the image
# Dropout drop the nodes in the layer that are less useful 
from keras import datasets 
from keras.datasets import fashion_mnist

# Properties of the dataset 
# 70,000 images of clothes(grayscale),size (28,28,1)

Train Test Split 

In [3]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

#storing the training data in a dataframe
train_df = pd.DataFrame(np.hstack([X_train.reshape(-1, 28*28),y_train.reshape(-1,1)]))


Discriminator Model

In [4]:


def build_discriminator(img_shape=(28,28,1),summary=True):
    model = Sequential()
    model.add(Input(shape=img_shape))
    model.add(Flatten()) #flatten() method is used for converting multi-dimensional array into one dimensional flatten array or say single dimensional array.
    model.add(Dense(512))  # 512 hidden units or nodes in a particular layer  # Hideen layer1
    model.add(LeakyReLU(0.3))  # LeakyReLU is a activation function  , 0.3 is the slope which is negative
    model.add(Dense(256))  # Hidden Layer 2
    model.add(LeakyReLU(0.3))
    model.add(Dense(1,activation='sigmoid'))  # Only 1 layer becoz it was for prediction of gans ,i.e true or false
    
    if summary:
        model.summary() 
    return model 
    

In [5]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',optimizer=Adam(0.0002, 0.5),metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trai

Generator Model

In [6]:
def build_generator(z_dim=(100),output_shape=(28,28),summary=True): # z_dim gives the size of the noise vector
    model = Sequential()
    model.add(Input(shape=z_dim))
    model.add(Dense(256))
    model.add(LeakyReLU(0.3))
    model.add(BatchNormalization(momentum=0.8)) # batchnormalization takes the output from previous alyer and normalizes and sends to next input layer
    model.add(Dense(512))
    model.add(LeakyReLU(0.3))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(0.3))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(output_shape),activation='tanh'))
    model.add(Reshape(output_shape))
    
    if summary :
        model.summary()
    return model 
    

In [ ]:
generator = build_generator()
noise = tf.random.normal([1,100])
generated_image = generator(noise,training=False)
#plt.imshow(generated_image[0,:,:,0],cmap='gray') # this prints the noise image 

GAN Model

In [7]:
def make_gan_model(generator, discriminator):
    model = tf.keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [10]:
discriminator = build_discriminator()
generator = build_generator()

Gan_Model = make_gan_model(generator, discriminator)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_12 (Dense)            (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-tr

Training The GAN 

In [11]:
def train(generator=None,discriminator=None,gan_model=None,epochs=50,batch_size=128,z_dim=100,sample_interval=200):
    (X_train, _), (_, _) = datasets.fashion_mnist.load_data()
    X_train = X_train /127.5  # Normalizing 
    
    
    
    
    for i in range(epochs):
        noise = np.random.normal(0,1,(batch_size,z_dim))
        
        discriminator.compile(loss='binary_crossentropy',optimizer=Adam(0.0002, 0.5),metrics=['accuracy'])

        
        real_imgs = X_train[np.random.randint(0,train_df.shape[0],batch_size)]
        print(real_imgs.shape)
        fake_imgs = generator.predict(noise)
        print(fake_imgs.shape)
        
        data = np.concatenate((fake_imgs,real_imgs),axis=0)
        labels = np.concatenate((np.zeros((batch_size,1)),np.ones((batch_size,1))),axis=0)
         
       
        
        disc_loss,disc_acc = discriminator.train_on_batch(data,labels)
        
        labels = np.ones((batch_size,1))   # Labels 
        #discriminator_loss = 0.5 *np.add(disc_loss_real,disc_loss_fake) 
        gen_loss = gan_model.train_on_batch(noise,labels)
        
        # training updates 
        print("%d [Discriminator Loss: %f,acc.: %.2f%%] [Generator Loss: %f]" % (epochs,disc_loss,100*disc_acc,gen_loss))
         
        # if epochs % sample_interval == 0:
        #     sample_images(epochs,generator)
    

In [12]:
# Noise for generator
z_dim = 100
z = Input(shape=(z_dim,))   # Input 
img = generator(z)

# Fix the discriminator
discriminator.trainable = False

# Get discriminator output
validity = discriminator(img)   # Output 

# Stack discriminator on top of generator
gan_model = Model(z, validity)
gan_model.compile(loss='binary_crossentropy', optimizer=Adam(0.0001, 0.5))
gan_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_4 (Sequential)   (None, 28, 28)            1493520   
                                                                 
 sequential_3 (Sequential)   (None, 1)                 533505    
                                                                 
Total params: 2,027,025
Trainable params: 1,489,936
Non-trainable params: 537,089
_________________________________________________________________


In [17]:
train(generator,discriminator,gan_model,epochs=100,batch_size=32 , sample_interval=200)


(32, 28, 28)
1/1 [==============================] - 0s 67ms/step
(32, 28, 28)
100 [Discriminator Loss: 1.152339,acc.: 28.12%] [Generator Loss: 0.235557]
(32, 28, 28)
1/1 [==============================] - 0s 38ms/step
(32, 28, 28)
100 [Discriminator Loss: 1.208279,acc.: 32.81%] [Generator Loss: 0.210897]
(32, 28, 28)
1/1 [==============================] - 0s 31ms/step
(32, 28, 28)
100 [Discriminator Loss: 1.204584,acc.: 29.69%] [Generator Loss: 0.205780]
(32, 28, 28)
1/1 [==============================] - 0s 32ms/step
(32, 28, 28)
100 [Discriminator Loss: 1.248197,acc.: 29.69%] [Generator Loss: 0.190712]
(32, 28, 28)
1/1 [==============================] - 0s 40ms/step
(32, 28, 28)
100 [Discriminator Loss: 1.248611,acc.: 23.44%] [Generator Loss: 0.195674]
(32, 28, 28)
1/1 [==============================] - 0s 24ms/step
(32, 28, 28)
100 [Discriminator Loss: 1.271092,acc.: 25.00%] [Generator Loss: 0.198236]
(32, 28, 28)
1/1 [==============================] - 0s 32ms/step
(32, 28, 28)
100 

In [15]:
batch_size=32
noise = np.random.normal(0,1,(batch_size,z_dim))
gen_imgs = generator.predict(noise)

1/1 [==============================] - 0s 24ms/step


Saving The data 

In [18]:
test_df = pd.DataFrame(gen_imgs.reshape(32,28*28))
test_df.to_csv('test.csv', index=False, header=False)